In [ ]:
import pandas as pd
import tomllib

from data.data_processing import select_rename_columns

In [ ]:
CONFIG_PATH = './data/config.toml'
def load_config(section_name: str) -> dict:
    with open(CONFIG_PATH, 'rb') as config_file:
        return tomllib.load(config_file)[section_name]

In [ ]:
df = pd.read_csv('./data/raw_data/pop_census/base-cc-serie-historique-2020.CSV', sep=';', dtype='object')

def population_preprocessing(df: pd.DataFrame, table: str='statistiques_pop') -> pd.DataFrame:
    population_config = load_config(table)

    df = df.copy()[~df['CODGEO'].str.startswith('97')]

    rename_mapper = {key: value for key, value in population_config.items() if isinstance(value, str)}
    df = df[list(population_config.keys())].rename(
        columns=rename_mapper
    )
    
    temp_dfs = []
    statistics_mapper = {key: value for key, value in population_config.items() if isinstance(value, list)}
    nbr_max_params = sum(map(len, statistics_mapper.values()))
    for column, fields in statistics_mapper.items():
        temp_dfs.append(
            pd.DataFrame(
                {name: df[name] for name in rename_mapper.values()} | {
                    'annee_debut': fields[1],
                    'annee_fin': fields[1] if len(fields) < nbr_max_params else fields[2],
                    'type_statistique': fields[0],
                    'valeur': df[column].astype(float)
                }
            )
        )

    return pd.concat(temp_dfs, ignore_index=True)

population_preprocessing(df)

In [ ]:
df = pd.read_csv('./data/raw_data/area/v_commune_2023.csv', dtype='object')
df1 = pd.read_csv('./data/raw_data/pop_census/base-cc-serie-historique-2020.CSV', sep=';', dtype='object')


df1[~df1['CODGEO'].isin(df[df['TYPECOM'] == 'ARM']['COM'])]

In [ ]:
print(('(\'' + pd.DataFrame({'a': ['Je suis', 'une', 'banante']})['a'] + '\', \'population\', \'2024\', \'2024\', 1)').str.cat(sep=',\n') + ";")

In [ ]:
file_path, table = f'./data/raw_data/area/v_commune_2023.csv', 'commune'
df = pd.read_csv(file_path) # Lecture du fichier CSV
df = df[df['TYPECOM'].isin(['COM', 'ARM'])] # Filtration des données
df = df[df['COM'].map(lambda x: int(x) < 97000 if x.isnumeric() else True)]
arm_com = df[df['TYPECOM'] == 'ARM']['COM'] # Commune qui sont des arrondissements
df = df.merge(
    pd.read_csv('./data/raw_data/pop_census/base-cc-serie-historique-2020.CSV', sep=';', dtype='object')[['CODGEO', 'SUPERF']],
    left_on='COM',
    right_on='CODGEO')
df = select_rename_columns(df, table) # Sélection et renommage des colonnes d'intérêts
df

In [ ]:
pd.read_csv('./data/raw_data/pop_census/base-cc-serie-historique-2020.CSV', sep=';', dtype='object')[['CODGEO', 'SUPERF']]